# Import modules

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from collections import Counter
import datetime

In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
# ex: from src.create_model_pickle import *


# Import data

In [ ]:
# Use our json to dataframe conversion class
from src.pipeline import pipeline_json
pj = pipeline_json('../data/data.json')

In [ ]:
df = pj.convert_to_df(scaling=True, filtered=True)
df.info()

In [ ]:
df.describe()

In [ ]:
df['event_life'] = df['event_life'].map(lambda x: 0 if np.isnan(x) else x)
df.info()

date_cols = ['approx_payout_date', 'event_created', 'event_created', 'event_end', \
                     'event_published', 'event_start', 'user_created']

for col in date_cols:
    print df[col][:5] - df['user_created'][:5]

In [ ]:
df['has_header'][:10]
    

a = df['event_published'][:5] - df['user_created'][:5]
a.var

i=12
total_payout = 0
payout_count = len(df['previous_payouts'][i])
payees = set()
for d in df['previous_payouts'][i]:
    total_payout += d['amount']
    payees.add(d['name'])
print total_payout, payout_count, payees

In [ ]:
a = set()

In [ ]:
df['ticket_types'][0][0].keys()

In [ ]:
Counter(df['fraud'])

In [ ]:
col = 'ticket_sales_amount'
print col
print "fraud :", df[df['fraud'] == True][col].mean()
print "not fraud :", df[df['fraud'] == False][col].mean()

In [ ]:
col = 'ticket_sales_count'
print col
print "fraud :", df[df['fraud'] == True][col].mean()
print "not fraud :", df[df['fraud'] == False][col].mean()

In [ ]:

col = 'body_length'
print col
print "fraud :", df[df['fraud'] == True][col].mean()
print "not fraud :", df[df['fraud'] == False][col].mean()

In [ ]:
df['short_description'][:10]

In [ ]:
df['body_length'][:10]

In [ ]:
Counter(df['currency'])

In [ ]:
# table = pivot_table(df, values='D', index=['A', 'B'],
# ...                     columns=['C'], aggfunc=np.sum)
table = pd.pivot_table(df, columns=['currency'], values=['fraud'], aggfunc=np.mean)
print table

In [ ]:
Counter(df['currency'])

In [ ]:
col = 'ticket_sales_events'
print col
print "fraud :", (df[df['fraud'] == True]['event_created'] - df[df['fraud'] == True]['user_created']).mean()
print "not fraud :", (df[df['fraud'] == False]['event_created'] - df[df['fraud'] == False]['user_created']).mean()


In [ ]:
col = 'has_analytics'
print col
print "fraud :", (df[df['fraud'] == True][col]).mean()
print "not fraud :", (df[df['fraud'] == False][col]).mean()


In [ ]:
import bs4

In [ ]:
df['event_life'].map(lambda x: 0 if np.isnan(x))

In [ ]:
df.info()

In [ ]:
df['event_life'][:10]

# Setup PostgreSQL table

PostgreSQL database is already setup with a table "events".

Table "events" has two fields :

    "predict" : [float]
    
    "json" : [text]
    

In [ ]:
#Read sample data to json file for testing writing to database
with open('../data/example.json', 'r') as f:
        json_ex = f.read()

In [ ]:
import psycopg2

In [ ]:
#Call prediction on JSON data

predict = 0.5

In [ ]:
#Write prediction and JSON string to database
conn = psycopg2.connect(dbname = 'fraud', port=5432, password='', user='wallace', host='localhost')
cur = conn.cursor()
cur.execute("INSERT INTO events (predict, json) VALUES (%s, %s)",(predict, json_ex))
conn.commit()
cur.close()
conn.close()

In [ ]:
#Read table and print results
conn = psycopg2.connect(dbname = 'fraud', port=5432, password='', user='wallace', host='localhost')
cur = conn.cursor()
query = '''SELECT * FROM events;'''
cur.execute(query)
rows = cur.fetchall()
for i in xrange(len(rows)):
    print rows[i]
    print
cur.close()
conn.close()


In [ ]:
cur.close()
conn.close()


# Read and split JSON file from disk

In [3]:
json_df = pd.read_json('../data/data.json')

In [ ]:
print len(json_df)

In [27]:
json_ex1 = json_df.iloc[0,:].to_json()
json_ex2 = json_df.iloc[1,:].to_json()
# my_json=pd.concat([json_ex1, json_ex2], axis=1).to_frame()
# my_json = .to_json()
# a = pd.read_json(my_json)
# my_json.head()
# a = pd.read_json('{' + json_ex1 + ',' + json_ex2 + '}')
print json_ex1
print
print json_ex2

{"acct_type":"fraudster_event","approx_payout_date":1266062400,"body_length":3852,"channels":5,"country":"US","currency":"USD","delivery_method":0.0,"description":"<p><a href=\"http:\/\/s432.photobucket.com\/albums\/qq49\/digusonline\/?action=view&amp;current=supersunday.jpg\" target=\"_blank\"><img src=\"http:\/\/i432.photobucket.com\/albums\/qq49\/digusonline\/supersunday.jpg\" border=\"0\" alt=\"Photobucket\" \/><\/a><\/p>\r\n<p>\u00a0<\/p>\r\n<p style=\"text-align: center;\"><font size=\"3\"><strong>Party Starz Entertaintment &amp; Diverse International Group Presents...<br \/><\/strong><\/font><br \/><font face=\"tahoma,arial,helvetica,sans-serif\" size=\"4\"><strong>The Official\u00a0\"99 Hour No Sleep\" Super Bowl <\/strong><\/font><\/p>\r\n<p style=\"text-align: center;\"><font face=\"tahoma,arial,helvetica,sans-serif\" size=\"4\"><strong>Weekend <\/strong><\/font><font face=\"tahoma,arial,helvetica,sans-serif\" size=\"4\"><strong>Grand Finale<\/strong><\/font><\/p>\r\n<p><br \

In [ ]:
b = pd.DataFrame(a)
print b

In [ ]:
from src.pipeline import pipeline_json
pj = pipeline_json(my_json)
X = pj.convert_to_df(scaling=True, filtered=True)
y = pj.output_labelarray()

In [ ]:
date_cols = ['approx_payout_date', 'event_created', 'event_created', 'event_end', \
             'event_published', 'event_start', 'user_created']
for col in date_cols:
    a[col] = pd.to_datetime(a[col], unit='s')
    
a

In [ ]:
a['acct_type']


TypeError: expected string or buffer

In [37]:
print response.content


{
  "approx_payout_date": 1358564400, 
  "body_length": 1029, 
  "channels": 12, 
  "country": "US", 
  "currency": "USD", 
  "delivery_method": 0.0, 
  "description": "<p><span><span class=\"fsl\">Join us for a day of community service, lunch, and a Broadway show! <br /> <br /> Details: <br /> <br /> 11:15 AM @ Dorot <br /> <br /> Meet at DOROT at 11:15am to distribute gifts to the elderly. <br /> 171 West 85th Street (Near Amsterdam Avenue)<br /> <br /> Group Lunch @ Dorot<br /> <br /> 3pm- Performance of My Name Is Asher Lev <br /> Venue: Westside Theatre<br /> Location: 407 W. 43rd St.<br /> <span style=\"font-size: large;\"><strong>$15 +  fees</strong></span><br /> w/ a Q&amp;A after the show <br /> <br /> </span></span></p>\r\n<p><span><span class=\"fsl\"><strong>This tickets will only be distributed at Dorot.</strong> <br /></span></span></p>\r\n<p><span><span class=\"fsl\"><br /></span></span></p>\r\n<p><span><span class=\"fsl\">For more information and to reserve a ticket as w

In [33]:
from src.pipeline import pipeline_json
# pj = pipeline_json(my_json)
# X = pj.convert_to_df(scaling=True, filtered=True)
# y = pj.output_labelarray()
pj = pipeline_json(response.content)
X = pj.convert_to_df(scaling=True, filtered=True)

In [34]:
X = pj.convert_to_df(scaling=True, filtered=True)

KeyError: 'acct_type'

In [40]:
from src.request_json import get_json
s = get_json()

In [41]:
print s


{
  "approx_payout_date": 1331530200, 
  "body_length": 7595, 
  "channels": 12, 
  "country": "CA", 
  "currency": "CAD", 
  "delivery_method": 0.0, 
  "description": "<p><object width=\"600\" height=\"437\" data=\"http://www.youtube.com/v/ebrvC5lsNoo?version=3&amp;hl=en_US&amp;rel=0\" type=\"application/x-shockwave-flash\"><param name=\"allowFullScreen\" value=\"true\" /><param name=\"allowscriptaccess\" value=\"always\" /><param name=\"src\" value=\"http://www.youtube.com/v/ebrvC5lsNoo?version=3&amp;hl=en_US&amp;rel=0\" /><param name=\"allowfullscreen\" value=\"true\" /></object></p>\r\n<p class=\"MsoNormal\" style=\"margin-bottom: .0001pt; mso-pagination: none; mso-layout-grid-align: none; text-autospace: none;\"><strong><span style=\"font-size: 11.0pt; mso-bidi-font-size: 15.0pt; font-family: Helvetica; mso-bidi-font-family: Helvetica;\"><br /></span></strong></p>\r\n<p class=\"MsoNormal\" style=\"margin-bottom: .0001pt; mso-pagination: none; mso-layout-grid-align: none; text-auto